In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import itertools

In [ ]:
raw_df = pd.read_csv('../data/accre-gpu-jobs-2022-v2.csv', on_bad_lines='skip')

In [ ]:
raw_df.columns = raw_df.columns.str.lower()

gpu_jobs_df = (raw_df
               .loc[raw_df['state'] == 'COMPLETED']
               .copy()
              )

gpu_jobs_df['reqtime'] = pd.to_timedelta(
    gpu_jobs_df['reqtime'].str.replace('-',' days ')
)

gpu_jobs_df['usedtime'] = pd.to_timedelta(
    gpu_jobs_df['usedtime'].str.replace('-',' days ')
)

In [ ]:
sns.boxplot(
    x = gpu_jobs_df['reqtime'] / np.timedelta64(1,'D')
);

In [ ]:
sns.boxplot(
    x = gpu_jobs_df['usedtime'] / np.timedelta64(1,'D')
);

In [ ]:
sns.histplot(
    x = gpu_jobs_df['reqtime'] / np.timedelta64(1,'D'),
    hue = gpu_jobs_df['partition'],
    bins = 10
);

In [ ]:
sns.histplot(
    x = gpu_jobs_df['usedtime'] / np.timedelta64(1,'D'),
    hue = gpu_jobs_df['partition'],
    bins = 10
);

In [ ]:
def all_used_percent(percent):
    values = []
    for x in percent:
        used = gpu_jobs_df.loc[
            gpu_jobs_df['usedtime'] >= gpu_jobs_df['reqtime'] * x
        ]['jobid'].count()
    
        per = (used / gpu_jobs_df['jobid'].count() * 100).round(2)
    
        values.append(per)
    return values

In [ ]:
percentages = [.1, .2]
days = ['less than', 1, 2, 3, 4, 5, 'more than']

In [ ]:
def less_than_day(percent):
    values = []
    for x in percent:
        used = gpu_jobs_df.loc[
                    ((gpu_jobs_df['reqtime'] < np.timedelta64(1,'D')) &
                    (gpu_jobs_df['usedtime'] >= gpu_jobs_df['reqtime'] * x)
                    )
                ]['jobid'].count()
            
        per = (
            (used / gpu_jobs_df.loc[
                    gpu_jobs_df['reqtime'] < np.timedelta64(1,'D')
            ]['jobid'].count() * 100
            )
        ).round(2)
        values.append(per)
    return values

In [ ]:
def per_day(day, percent):
    values = []
    for x in percent:
        used = gpu_jobs_df.loc[
                    ((gpu_jobs_df['reqtime'] >= np.timedelta64(day,'D')) &
                     (gpu_jobs_df['reqtime'] < np.timedelta64(day + 1,'D')) &
                    (gpu_jobs_df['usedtime'] >= gpu_jobs_df['reqtime'] * x)
                    )
                ]['jobid'].count()
        per = (
                (used / gpu_jobs_df.loc[
                    ((gpu_jobs_df['reqtime'] >= np.timedelta64(day,'D')) &
                     (gpu_jobs_df['reqtime'] < np.timedelta64(day + 1,'D'))
                    )
                ]['jobid'].count() * 100)
            ).round(2)
        values.append(per)
    return values

In [ ]:
def more_than_five(percent):
    values = []
    for x in percent:
        used = gpu_jobs_df.loc[
                ((gpu_jobs_df['reqtime'] > np.timedelta64(5,'D')) &
                (gpu_jobs_df['usedtime'] >= gpu_jobs_df['reqtime'] * x)
                )
            ]['jobid'].count()
        per = (
            (used / gpu_jobs_df.loc[
                gpu_jobs_df['reqtime'] > np.timedelta64(5,'D')
            ]['jobid'].count() * 100
            )
        ).round(2)
        values.append(per)
    return values

In [ ]:
percent_df = pd.DataFrame()

percent_df['percentages'] = percentages
percent_df['percentages'] = percent_df['percentages'] * 100

percent_df['all_entries'] = all_used_percent(percentages)

percent_df['less_than_day'] = less_than_day(percentages)

percent_df['one_day'] = per_day(1, percentages)
percent_df['two_days'] = per_day(2, percentages)
percent_df['three_days'] = per_day(3, percentages)
percent_df['four_days'] = per_day(4, percentages)
percent_df['five_days'] = per_day(5, percentages)

percent_df['more_than_five'] = more_than_five(percentages)

percent_df = percent_df.transpose().reset_index()
percent_df.columns = percent_df.iloc[0]
percent_df = (percent_df
              .drop(percent_df.index[0])
              .rename(columns = {'percentages' : 'req_days'})
             )

In [ ]:
percent_df

In [ ]:
req_2_more_df = gpu_jobs_df.loc[
    gpu_jobs_df['reqtime'] >= np.timedelta64(2,'D')
]

In [ ]:
def used_percent(data, percent):
    used = data.loc[
        data['usedtime'] >= data['reqtime'] * percent
    ]['jobid'].count()
    
    per = (used / data['jobid'].count() * 100).round(2)
    
    return per

In [ ]:
used_percent(req_2_more_df, .1)

In [ ]:
used_percent(req_2_more_df, .2)

In [ ]:
all_entries = (req_2_more_df.loc[
    req_2_more_df['usedtime'] <= np.timedelta64(1,'h')
]['jobid'].count() / req_2_more_df['jobid'].count() * 100).round(2)

In [ ]:
maxwell = (req_2_more_df.loc[
    (req_2_more_df['usedtime'] <= np.timedelta64(1,'h')) & (req_2_more_df['partition'] == 'maxwell')
]['jobid'].count() / req_2_more_df.loc[req_2_more_df['partition'] == 'maxwell']['jobid'].count() * 100).round(2)

In [ ]:
pascal = (req_2_more_df.loc[
    (req_2_more_df['usedtime'] <= np.timedelta64(1,'h')) & (req_2_more_df['partition'] == 'pascal')
]['jobid'].count() / req_2_more_df.loc[req_2_more_df['partition'] == 'pascal']['jobid'].count() * 100).round(2)

In [ ]:
turing = (req_2_more_df.loc[
    (req_2_more_df['usedtime'] <= np.timedelta64(1,'h')) & (req_2_more_df['partition'] == 'turing')
]['jobid'].count() / req_2_more_df.loc[req_2_more_df['partition'] == 'turing']['jobid'].count() * 100).round(2)

In [ ]:
under_hour_df =  pd.DataFrame()

under_hour_df['partition'] = ['all_entries', 'maxwell', 'pascal', 'turing']

under_hour_df['total'] = [
    req_2_more_df['jobid'].count(),
    req_2_more_df.loc[req_2_more_df['partition'] == 'maxwell']['jobid'].count(),
    req_2_more_df.loc[req_2_more_df['partition'] == 'pascal']['jobid'].count(),
    req_2_more_df.loc[req_2_more_df['partition'] == 'turing']['jobid'].count()
]

under_hour_df['under_hour'] = [
    req_2_more_df.loc[req_2_more_df['usedtime'] <= np.timedelta64(1,'h')]['jobid'].count(),
    (req_2_more_df.loc[
        (req_2_more_df['usedtime'] <= np.timedelta64(1,'h')) & (req_2_more_df['partition'] == 'maxwell')
    ]['jobid'].count()),
    (req_2_more_df.loc[
        (req_2_more_df['usedtime'] <= np.timedelta64(1,'h')) & (req_2_more_df['partition'] == 'pascal')
    ]['jobid'].count()),
    (req_2_more_df.loc[
        (req_2_more_df['usedtime'] <= np.timedelta64(1,'h')) & (req_2_more_df['partition'] == 'turing')
    ]['jobid'].count())
]

under_hour_df['percent'] = [all_entries, maxwell, pascal, turing]

In [ ]:
under_hour_df

In [ ]:
sns.barplot(
    data = under_hour_df,
    x = 'partition',
    y = 'total'
);

In [ ]:
top_ten_per_df = req_2_more_df.loc[req_2_more_df['usedtime'] >= req_2_more_df['reqtime'] * .1].copy()

In [ ]:
top_ten_per_df.loc[top_ten_per_df['partition'] == 'pascal']['jobid'].count() / top_ten_per_df['jobid'].count()

In [ ]:
top_ten_per_df.loc[top_ten_per_df['partition'] == 'maxwell']['jobid'].count() / top_ten_per_df['jobid'].count()

In [ ]:
top_ten_per_df.loc[top_ten_per_df['partition'] == 'turing']['jobid'].count() / top_ten_per_df['jobid'].count()

In [ ]:
top_ten_per_df['used_per_day'] = top_ten_per_df['usedtime'] / np.timedelta64(1,'D')
top_ten_per_df['req_per_day'] = top_ten_per_df['reqtime'] / np.timedelta64(1,'D')

top_ten_per_df['time_difference'] = top_ten_per_df['req_per_day'] - top_ten_per_df['used_per_day']

In [ ]:
five_day_run_out_df = top_ten_per_df.loc[
    (top_ten_per_df['reqtime'] == np.timedelta64(5, 'D')) & (top_ten_per_df['time_difference'] <= 0.0417)
]

In [ ]:
five_day_run_out_df.groupby(['account', 'user'])['jobid'].count().sort_values(ascending = False)

In [ ]:
five_day_run_out_df.groupby(['account', 'user','partition'])['jobid'].count().sort_values(ascending = False)

In [ ]:
five_day_run_out_df['jobid'].count()

In [ ]:
def total(account, user):
    count = gpu_jobs_df.loc[
        ((gpu_jobs_df['account'] == account) & 
         (gpu_jobs_df['user'] == user) )
    ]['jobid'].count()
    
    return count

In [ ]:
def five_hour_req(account, user):
    count = gpu_jobs_df.loc[
        ((gpu_jobs_df['account'] == account) & 
         (gpu_jobs_df['user'] == user) & 
         (gpu_jobs_df['reqtime'] == np.timedelta64(5,'D')))
    ]['jobid'].count()
    
    return count

In [ ]:
def under_hour(account, user):
    count = gpu_jobs_df.loc[
        ((gpu_jobs_df['account'] == account) & 
         (gpu_jobs_df['user'] == user) & 
         (gpu_jobs_df['reqtime'] == np.timedelta64(5,'D')) &
         ((gpu_jobs_df['usedtime'] <= np.timedelta64(1,'h'))))
    ]['jobid'].count()
    
    return count

In [ ]:
def percentiles(account, user):
    count = (gpu_jobs_df.loc[
        ((gpu_jobs_df['account'] == account) & 
         (gpu_jobs_df['user'] == user) &
         (gpu_jobs_df['reqtime'] == np.timedelta64(5,'D'))
        )
    ]['usedtime']
             .describe(percentiles=[.01, .05, .1, .25, .5, .75, .9, .95, .99])
            )[4:13].to_frame().reset_index().transpose()
    count.columns = count.iloc[0]
    count = count.drop(count.index[0]).reset_index().drop(columns = ['index'])
    return count

In [ ]:
percentiles('harovin', 'porsha')

In [ ]:
har_total = total('harovin', 'porsha')

har_five = five_hour_req('harovin', 'porsha')

har_under_hour = under_hour('harovin', 'porsha')

In [ ]:
gla_total = total('glasshouse', 'jolette')

gla_five = five_hour_req('glasshouse', 'jolette')

gla_under_hour = under_hour('glasshouse', 'jolette')

In [ ]:
val_total = total('harovin', 'valentina')

val_five = five_hour_req('harovin', 'valentina')

val_under_hour = under_hour('harovin', 'valentina')

In [ ]:
eri_total = total('glasshouse', 'erick')

eri_five = five_hour_req('glasshouse', 'erick')

eri_under_hour = under_hour('glasshouse', 'erick')

In [ ]:
two_run_out_df = pd.DataFrame()

two_run_out_df['account'] = ['harovin', 'glasshouse', 'harovin']

two_run_out_df['user'] = ['porsha', 'jolette', 'valentina']

two_run_out_df['total_jobs'] = [har_total, gla_total, val_total]

two_run_out_df['five_day_total'] = [har_five, gla_five, val_five]

two_run_out_df['finish_under_hour'] = [
    (har_under_hour/har_five * 100).round(2), 
    (gla_under_hour/gla_five * 100).round(2), 
    (val_under_hour/val_five * 100).round(2)
]

#two_run_out_df['1%'] = [
    #percentiles('harovin', 'porsha')['1%'][0], 
    #percentiles('glasshouse', 'jolette')['1%'][0], 
    #percentiles('harovin', 'valentina')['1%'][0]
#]

two_run_out_df['5%'] = [
    percentiles('harovin', 'porsha')['5%'][0], 
    percentiles('glasshouse', 'jolette')['5%'][0], 
    percentiles('harovin', 'valentina')['5%'][0]
]

#two_run_out_df['10%'] = [
    #percentiles('harovin', 'porsha')['10%'][0], 
    #percentiles('glasshouse', 'jolette')['10%'][0], 
    #percentiles('harovin', 'valentina')['10%'][0]
#]

two_run_out_df['25%'] = [
    percentiles('harovin', 'porsha')['25%'][0], 
    percentiles('glasshouse', 'jolette')['25%'][0], 
    percentiles('harovin', 'valentina')['25%'][0]
]

two_run_out_df['50%'] = [
    percentiles('harovin', 'porsha')['50%'][0], 
    percentiles('glasshouse', 'jolette')['50%'][0], 
    percentiles('harovin', 'valentina')['50%'][0]
]

two_run_out_df['75%'] = [
    percentiles('harovin', 'porsha')['75%'][0], 
    percentiles('glasshouse', 'jolette')['75%'][0], 
    percentiles('harovin', 'valentina')['75%'][0]
]

#two_run_out_df['90%'] = [
    #percentiles('harovin', 'porsha')['90%'][0], 
    #percentiles('glasshouse', 'jolette')['90%'][0], 
    #percentiles('harovin', 'valentina')['90%'][0]
#]

two_run_out_df['95%'] = [
    percentiles('harovin', 'porsha')['95%'][0], 
    percentiles('glasshouse', 'jolette')['95%'][0], 
    percentiles('harovin', 'valentina')['95%'][0]
]

#two_run_out_df['99%'] = [
    #percentiles('harovin', 'porsha')['99%'][0], 
    #percentiles('glasshouse', 'jolette')['99%'][0], 
    #percentiles('harovin', 'valentina')['99%'][0]
#]

two_run_out_df['five_day_timeout'] = [
    (273 / har_five * 100).round(2), 
    (213 / gla_five * 100).round(2), 
    (50 / val_five * 100).round(2)
]

In [ ]:
two_run_out_df

In [ ]:
two_run_out_df.to_excel("../data/close_calls.xlsx")